<!-- [ 2022. 09. 05] - 과제 -

(1) 전처리
- 나라별 알파벳 패턴(많이 사용되는 알파벳) 분석
-

(2) 학습
- 교차 검증 적용
- 분류 모델 적용 학습 -->

# [ 2022. 09. 05] - 과제 -

### (1) 전처리
- 나라별 알파벳 패턴(많이 사용되는 알파벳) 분석


### (2) 학습
- 교차 검증 적용
- 분류 모델 적용 학습

## [1] 데이터 로딩 및 전처리
---

## train

In [639]:
# 모듈로딩
import pandas as pd
import os
import re
import numpy as np
from sklearn.utils import all_estimators
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

In [640]:
# 경고 무시
import warnings
warnings.filterwarnings(action='ignore')

In [641]:
# 파일 관련
FILE_train=os.listdir('./lang/train')
FILE_test=os.listdir('./lang/test')

In [642]:
# 알파벳 빈도 수 체크
count=[]
for file in FILE_train:
    f=open('./lang/train/'+str(file), 'r', encoding = 'utf-8')  # 파일명의 텍스트 파일을 읽기전용으로 열기
    text = f.read()  # 읽어온 텍스트파일의 전체를 하나의 문자열로 읽기
    text = text.lower()  # 텍스트파일 전체를 소문자로 바꾸기
    f.close()
    r_text=re.sub("[^a-z]","",text)
    
    Alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    d = [0] * 26
    
    for i in r_text:
        if i in Alphabet:
            idx = Alphabet.find(i)
            d[idx] += 1
            
    count.append(d)

# 알파벳 빈도수 리스트 => 시리즈 형변환
b=[]
for i in count:
    i=pd.Series(i)
    b.append(i)
    
# 텍스트 파일 빈도 수 => 데이터프레임 형변환
train_df = pd.concat(b, axis = 1).T
train_df.columns = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# 값을 파일별로 퍼센트로 넣기
country_sum = train_df.apply(np.sum,axis = 1)

for i in train_df.index:
    row=[]
    for j in train_df.columns:
        row.append((train_df.iloc[i][j] / country_sum[i])*100)
    train_df.iloc[i]=row

# 나라 컬럼 넣기
idx = []
for i in FILE_train:
    idx.append(i[:2])
    
train_df.insert(0, 'country', idx)

## test

In [643]:
# 알파벳 빈도 수 체크
count=[]
for file in FILE_test:
    f=open('./lang/test/'+str(file), 'r', encoding = 'utf-8')  # 파일명의 텍스트 파일을 읽기전용으로 열기
    text = f.read()  # 읽어온 텍스트파일의 전체를 하나의 문자열로 읽기
    text = text.lower()  # 텍스트파일 전체를 소문자로 바꾸기
    f.close()
    r_text=re.sub("[^a-z]","",text)
    
    Alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    d = [0] * 26
    
    for i in r_text:
        if i in Alphabet:
            idx = Alphabet.find(i)
            d[idx] += 1
            
    count.append(d)

# 알파벳 빈도수 리스트 => 시리즈 형변환
b=[]
for i in count:
    i=pd.Series(i)
    b.append(i)
    
# 텍스트 파일 빈도 수 => 데이터프레임 형변환
test_df = pd.concat(b, axis = 1).T
test_df.columns = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# 값을 파일별로 퍼센트로 넣기
country_sum = test_df.apply(np.sum,axis = 1)

for i in test_df.index:
    row=[]
    for j in test_df.columns:
        row.append((test_df.iloc[i][j] / country_sum[i])*100)
    test_df.iloc[i]=row

# 나라 컬럼 넣기
idx = []
for i in FILE_test:
    idx.append(i[:2])
    
test_df.insert(0, 'country', idx)

In [644]:
print(train_df.head(), test_df.head())

  country         a         b         c         d          e         f  \
0      en  7.595212  1.284004  4.570185  4.613711  10.533188  1.566921   
1      en  8.417789  1.991177  3.040420  3.886968  13.699773  1.740789   
2      en  7.164592  1.217151  4.564315  3.264177  12.005533  1.466113   
3      en  7.220965  2.771493  2.997738  3.959276  12.104072  1.677979   
4      en  7.380640  2.036794  3.109943  3.964082  14.126150  2.036794   

          g         h         i  ...         q         r         s         t  \
0  1.915125  4.374320  7.399347  ...  0.000000  7.769314  6.137106  8.052231   
1  3.123882  2.742339  7.535472  ...  0.548468  9.013950  7.165852  7.773936   
2  2.517289  2.351314  9.460581  ...  0.165975  5.394191  8.796680  8.105118   
3  2.356712  5.901207  6.523379  ...  0.037707  5.920060  7.334087  9.370287   
4  2.036794  5.694262  6.504599  ...  0.043802  7.249233  5.957074  9.548839   

          u         v         w         x         y         z  
0  2.58977

## [2] 학습 데이터 분리
---

In [645]:
train_input = train_df[train_df.columns[1:]]
train_target = train_df['country']

test_input = test_df[test_df.columns[1:]]
test_target = test_df['country']

In [646]:
# DataFrame -> Numpy
train_input = train_input.to_numpy()
test_input = test_input.to_numpy()

train_input.shape, test_input.shape

((20, 26), (8, 26))

## [3] 학습 및 평가
---

### sklearn 모델 추천 함수 활용

In [647]:
models = all_estimators(type_filter='classifier')

In [648]:
scores = []
for name, model in models:
    try:
        # 모델 객체 생성
        md=model()
        # 학습
        md.fit(train_input, train_target)
        # 평가
        result=md.score(test_input, test_target)
        scores.append((name, np.round(result, 3)))
    except:
        pass
    
scores

[('AdaBoostClassifier', 0.75),
 ('BaggingClassifier', 1.0),
 ('BernoulliNB', 0.25),
 ('CalibratedClassifierCV', 0.875),
 ('ComplementNB', 0.75),
 ('DecisionTreeClassifier', 0.75),
 ('DummyClassifier', 0.25),
 ('ExtraTreeClassifier', 0.625),
 ('ExtraTreesClassifier', 1.0),
 ('GaussianNB', 0.75),
 ('GaussianProcessClassifier', 1.0),
 ('GradientBoostingClassifier', 1.0),
 ('HistGradientBoostingClassifier', 0.25),
 ('KNeighborsClassifier', 0.75),
 ('LabelPropagation', 0.875),
 ('LabelSpreading', 0.875),
 ('LinearDiscriminantAnalysis', 0.875),
 ('LinearSVC', 1.0),
 ('LogisticRegression', 1.0),
 ('LogisticRegressionCV', 1.0),
 ('MLPClassifier', 0.875),
 ('MultinomialNB', 1.0),
 ('NearestCentroid', 1.0),
 ('NuSVC', 1.0),
 ('PassiveAggressiveClassifier', 1.0),
 ('Perceptron', 1.0),
 ('QuadraticDiscriminantAnalysis', 0.625),
 ('RandomForestClassifier', 1.0),
 ('RidgeClassifier', 0.875),
 ('RidgeClassifierCV', 0.875),
 ('SGDClassifier', 1.0),
 ('SVC', 1.0)]

### LogisticRegression

In [649]:
model = LogisticRegression()
model.fit(train_input, train_target)

LogisticRegression()

In [650]:
print('train set score: ', model.score(train_input, train_target), '\ntest set score: ', model.score(test_input, test_target))

train set score:  1.0 
test set score:  1.0


In [651]:
# 5-Fold로 5등분으로 나누어서 학습/검증 모델 생성 진행
result=cross_validate(model,
                     train_input,
                     train_target,
                     return_train_score=True)
print('train_score: ', np.mean(result['train_score']), '\ntest_score: ', np.mean(result['test_score']))

train_score:  1.0 
test_score:  1.0


### SGDClassifier

In [652]:
sc = SGDClassifier()
sc.fit(train_input, train_target)

SGDClassifier()

In [653]:
print('train set score: ', sc.score(train_input, train_target), '\ntest set score: ', sc.score(test_input, test_target))

train set score:  1.0 
test set score:  1.0


In [654]:
# 5-Fold로 5등분으로 나누어서 학습/검증 모델 생성 진행
result=cross_validate(sc,
                     train_input,
                     train_target,
                     return_train_score=True)
print('train_score: ', np.mean(result['train_score']), '\ntest_score: ', np.mean(result['test_score']))

train_score:  0.95 
test_score:  0.9


### SVM(Support Vector Machine)
---

In [655]:
svm_clf = SVC()
svm_clf.fit(train_input, train_target)

SVC()

In [656]:
print('train set score: ', svm_clf.score(train_input, train_target), '\ntest set score: ', svm_clf.score(test_input, test_target))

train set score:  1.0 
test set score:  1.0


In [657]:
# 5-Fold로 5등분으로 나누어서 학습/검증 모델 생성 진행
result=cross_validate(svm_clf,
                     train_input,
                     train_target,
                     return_train_score=True)
print('train_score: ', np.mean(result['train_score']), '\ntest_score: ', np.mean(result['test_score']))

train_score:  1.0 
test_score:  1.0


### 결정트리(Decision Tree)
---

In [658]:
dtModel = DecisionTreeClassifier()
dtModel.fit(train_input, train_target)

DecisionTreeClassifier()

In [659]:
print('train set score: ', dtModel.score(train_input, train_target), '\ntest set score: ', dtModel.score(test_input, test_target))

train set score:  1.0 
test set score:  0.875


In [660]:
# 5-Fold로 5등분으로 나누어서 학습/검증 모델 생성 진행
result=cross_validate(dtModel,
                     train_input,
                     train_target,
                     return_train_score=True)
print('train_score: ', np.mean(result['train_score']), '\ntest_score: ', np.mean(result['test_score']))

train_score:  1.0 
test_score:  0.75


### RandomForest

In [661]:
forestModel = RandomForestClassifier()
forestModel.fit(train_input, train_target)

RandomForestClassifier()

In [662]:
print('train set score: ', forestModel.score(train_input, train_target), '\ntest set score: ', forestModel.score(test_input, test_target))

train set score:  1.0 
test set score:  1.0


In [663]:
# 5-Fold로 5등분으로 나누어서 학습/검증 모델 생성 진행
result=cross_validate(forestModel,
                     train_input,
                     train_target,
                     return_train_score=True)
print('train_score: ', np.mean(result['train_score']), '\ntest_score: ', np.mean(result['test_score']))

train_score:  1.0 
test_score:  1.0


## [4] 테스트 => predict() : 새로운 데이터로 진행
---
- 영어 글귀로 예측해보기

In [664]:
# 알파벳 빈도 수 체크
f=open('./lang/pd.txt', 'r', encoding = 'utf-8')  # 파일명의 텍스트 파일을 읽기전용으로 열기
text = f.read()  # 읽어온 텍스트파일의 전체를 하나의 문자열로 읽기
text = text.lower()  # 텍스트파일 전체를 소문자로 바꾸기
f.close()
r_text=re.sub("[^a-z]","",text)

Alphabet = 'abcdefghijklmnopqrstuvwxyz'

d = [0] * 26

for i in r_text:
    if i in Alphabet:
        idx = Alphabet.find(i)
        d[idx] += 1

d = pd.Series(d)
d = (d / sum(d)) * 100
d = d.to_numpy()
d = d.reshape(1, -1)

In [665]:
print(model.predict(d))
print(sc.predict(d))
print(svm_clf.predict(d))
print(dtModel.predict(d))
print(forestModel.predict(d))

['en']
['en']
['en']
['fr']
['en']
